In [1]:
import pandas as pd
from typing import Tuple
import math
import numpy as np
from tqdm import tqdm

# Cleaning Game Info
This notebook provides a clean version of the raw `data/gameinfo.csv` file.
If does the following:
1. Filtering for competitive games.
2. Adding a 'homewon' column, that is True if the home team won and False otherwise.
3. Adding a 'timestamp' column, which provides a `pd.Timestamp` for the game start time.
4. Adding an 'visdistancetraveled' column, which contains the distance in miles from the visiting team's stadium to the home team's stadium.
5. Adding 'homerestdays' and 'visrestdays' columns, which contains the number of days between the current game and the previous game for the home and visiting teams.
6. Save to a new csv, `.data/gameinfo_clean.csv`

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
all_games = pd.read_csv('./gameinfo.csv')
all_games.head()

/var/folders/f3/w3vlb72x47z3vhjsm9s6jlxc0000gn/T/ipykernel_4746/2207742255.py:1: DtypeWarning: Columns (10,11,13,17,19,21,27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  all_games = pd.read_csv('./gameinfo.csv')


,gid,visteam,hometeam,site,date,number,starttime,daynight,innings,tiebreaker,usedh,htbf,timeofgame,attendance,fieldcond,precip,sky,temp,winddir,windspeed,oscorer,forfeit,suspend,umphome,ump1b,ump2b,ump3b,umplf,umprf,wp,lp,save,gametype,vruns,hruns,wteam,lteam,line,batteries,lineups,box,pbp,season
0,LS3189904140,CHN,LS3,LOU03,18990414,0.0,0:00PM,day,NaN,NaN,False,NaN,113.0,11500.0,unknown,unknown,unknown,0.0,unknown,-1.0,NaN,NaN,NaN,burno101,warna901,NaN,NaN,NaN,NaN,grifc101,cunnb103,NaN,regular,15,1,CHN,LS3,y,both,y,y,NaN,1899
1,PHI189904140,WSN,PHI,PHI09,18990414,0.0,0:00PM,day,NaN,NaN,False,NaN,120.0,12000.0,unknown,unknown,unknown,0.0,unknown,-1.0,NaN,NaN,NaN,huntj901,connt901,NaN,NaN,NaN,NaN,piatw101,killf101,NaN,regular,5,6,PHI,WSN,y,both,y,y,NaN,1899
2,BLN189904150,NY1,BLN,BAL07,18990415,0.0,0:00PM,day,NaN,NaN,False,NaN,130.0,3912.0,unknown,unknown,unknown,0.0,unknown,-1.0,NaN,NaN,NaN,emslb101,bettw901,NaN,NaN,NaN,NaN,kitsf101,dohee101,NaN,regular,3,5,BLN,NY1,y,both,y,y,NaN,1899
3,BRO189904150,BSN,BRO,NYC12,18990415,0.0,0:00PM,day,NaN,NaN,False,NaN,120.0,20167.0,unknown,unknown,unknown,0.0,unknown,-1.0,NaN,NaN,NaN,andre101,gaffj801,NaN,NaN,NaN,NaN,nichk101,kennb101,NaN,regular,1,0,BSN,BRO,y,both,y,y,NaN,1899
4,CIN189904150,PIT,CIN,CIN05,18990415,0.0,0:00PM,day,NaN,NaN,False,NaN,130.0,10000.0,unknown,unknown,unknown,0.0,unknown,-1.0,NaN,NaN,NaN,sware101,warna901,NaN,NaN,NaN,NaN,tannj101,hawlp101,NaN,regular,5,2,PIT,CIN,y,both,y,y,NaN,1899


## 1. Filter for Competitive Games

In [4]:
valid_games = ['regular', 'championship', 'worldseries', 'lcs',
                'playoff', 'divisionseries', 'wildcard']

all_games = all_games[all_games['gametype'].isin(valid_games)].reset_index(drop=True)

## 2. Add 'homewon' Column

In [5]:
all_games['homewon'] = list((all_games['hruns'] > all_games['vruns']))
all_games.head()

,gid,visteam,hometeam,site,date,number,starttime,daynight,innings,tiebreaker,usedh,htbf,timeofgame,attendance,fieldcond,precip,sky,temp,winddir,windspeed,oscorer,forfeit,suspend,umphome,ump1b,ump2b,ump3b,umplf,umprf,wp,lp,save,gametype,vruns,hruns,wteam,lteam,line,batteries,lineups,box,pbp,season,homewon
0,LS3189904140,CHN,LS3,LOU03,18990414,0.0,0:00PM,day,NaN,NaN,False,NaN,113.0,11500.0,unknown,unknown,unknown,0.0,unknown,-1.0,NaN,NaN,NaN,burno101,warna901,NaN,NaN,NaN,NaN,grifc101,cunnb103,NaN,regular,15,1,CHN,LS3,y,both,y,y,NaN,1899,False
1,PHI189904140,WSN,PHI,PHI09,18990414,0.0,0:00PM,day,NaN,NaN,False,NaN,120.0,12000.0,unknown,unknown,unknown,0.0,unknown,-1.0,NaN,NaN,NaN,huntj901,connt901,NaN,NaN,NaN,NaN,piatw101,killf101,NaN,regular,5,6,PHI,WSN,y,both,y,y,NaN,1899,True
2,BLN189904150,NY1,BLN,BAL07,18990415,0.0,0:00PM,day,NaN,NaN,False,NaN,130.0,3912.0,unknown,unknown,unknown,0.0,unknown,-1.0,NaN,NaN,NaN,emslb101,bettw901,NaN,NaN,NaN,NaN,kitsf101,dohee101,NaN,regular,3,5,BLN,NY1,y,both,y,y,NaN,1899,True
3,BRO189904150,BSN,BRO,NYC12,18990415,0.0,0:00PM,day,NaN,NaN,False,NaN,120.0,20167.0,unknown,unknown,unknown,0.0,unknown,-1.0,NaN,NaN,NaN,andre101,gaffj801,NaN,NaN,NaN,NaN,nichk101,kennb101,NaN,regular,1,0,BSN,BRO,y,both,y,y,NaN,1899,False
4,CIN189904150,PIT,CIN,CIN05,18990415,0.0,0:00PM,day,NaN,NaN,False,NaN,130.0,10000.0,unknown,unknown,unknown,0.0,unknown,-1.0,NaN,NaN,NaN,sware101,warna901,NaN,NaN,NaN,NaN,tannj101,hawlp101,NaN,regular,5,2,PIT,CIN,y,both,y,y,NaN,1899,False


## 3. Add 'timestamp' Column

In [6]:
def get_hms(raw_starttime: str) -> Tuple[int, int]:
    """For the given raw start time (e.g. '5:30PM'), returns the hour and minute.
    
    If raw_starttime is nan, this will return 0, 0.
    
    If there is no 'AM' or 'PM' (e.g., '' or '?M'), it will assume a 24 hour clock.
    """

    if pd.isna(raw_starttime):
        return 0, 0
    
    col_idx = raw_starttime.find(':')
    h = int(raw_starttime[:col_idx]) 
    m = int(raw_starttime[col_idx + 1:col_idx + 3])
    
    # If AM or PM, need to do some more conversions
        
    # If time is PM and not 12:00, add 12 hours
    # Note some entries in all_games are 0:00 PM - this would also correctly add 12 hours,
    # making it the familiar 12:00 PM
    if raw_starttime[-2:].upper() == 'PM' and h != 12:
        h += 12
            
    elif raw_starttime[-2:].upper() == 'AM' and h == 12: # Edge case - if midnight, h should be 0
        h = 0
        
    return h, m

def get_timestamp(game: pd.DataFrame) -> pd.Timestamp:
    """For a single row, gets its game start timestamp."""
    #print(game['starttime'])
    h, min = get_hms(game['starttime'])
    
    raw_date = str(game['date'])

    y = int(raw_date[:4])
    mon = int(raw_date[4:6])
    d = int(raw_date[6:])
    return pd.Timestamp(year=y, month=mon, day=d, hour=h, minute=min)

In [ ]:
# Add the timestamp column

all_games['timestamp'] = all_games.apply(get_timestamp, axis=1)
all_games.head()

## 4. Add 'visdistancetraveled' Column

In [ ]:
# Add temporary latitude and longitude columns
parks = pd.read_csv('Parks.csv')
all_games = pd.merge(all_games, parks[['PARKID', 'Latitude', 'Longitude']], how='left', left_on='site', right_on='PARKID').reset_index(drop=True)
all_games = all_games.drop('PARKID', axis=1)

In [ ]:
def haversine(lat1, lon1, lat2, lon2):
    """Returns Haversine distance between two pairs of latitudes and longitudes."""
    R = 3958.8  # Earth radius in miles
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)
    a = math.sin(dphi / 2)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(dlambda / 2)**2
    return 2 * R * math.atan2(math.sqrt(a), math.sqrt(1 - a))

In [ ]:
away_dists = []
n_games = all_games.shape[0]
for i, game in tqdm(all_games.iterrows()):
    
    cur_lat = game['Latitude']
    cur_lon = game['Longitude']
    
    away_team = game['visteam']
    
    # Get lat and lon of the closest home game they played
    before = i - 1
    after = i + 1
    while before >= 0 or after <= n_games - 1:
        if before >= 0:
            before_gm = all_games.iloc[before]
            if before_gm['hometeam'] == away_team:
                away_lat = before_gm['Latitude']
                away_lon = before_gm['Longitude']
                break
            
        if after <= n_games - 1:
            after_gm = all_games.iloc[after]
            if after_gm['hometeam'] == away_team:
                away_lat = after_gm['Latitude']
                away_lon = after_gm['Longitude']
                break
        
        if before >= 0:   
            before -= 1
        if after <= n_games - 1:
            after += 1

    else: # We never saw them play home, so we have no idea where they play - leave na.
        away_dists.append(np.nan)
        continue
        
    away_dists.append(haversine(away_lat, away_lon, cur_lat, cur_lon))
    
all_games['visdistancetraveled'] = away_dists

218699it [08:18, 438.41it/s]


## 5. Add Rest Day Columns

### Home Rest Days

In [ ]:
home_rest_days = []
away_rest_days = []

last_played = {}

for _, game in tqdm(all_games.iterrows()):
    home_team = game['hometeam']
    away_team = game['visteam']
    timestamp = game['timestamp']
    
    prev_home_t = last_played.get(home_team)
    prev_away_t = last_played.get(away_team)
    
    home_rest_days.append((timestamp.floor('D') - prev_home_t.floor('D')).days if prev_home_t is not None else np.nan)
    away_rest_days.append((timestamp.floor('D') - prev_away_t.floor('D')).days if prev_away_t is not None else np.nan)

    last_played[home_team] = timestamp
    last_played[away_team] = timestamp
 
all_games['homerestdays'] = home_rest_days
all_games['visrestdays'] = away_rest_days

0it [00:00, ?it/s]

218699it [03:39, 997.98it/s] 


In [ ]:
all_games.tail(20)

,gid,visteam,hometeam,site,date,number,starttime,daynight,innings,tiebreaker,usedh,htbf,timeofgame,attendance,fieldcond,precip,sky,temp,winddir,windspeed,oscorer,forfeit,suspend,umphome,ump1b,ump2b,ump3b,umplf,umprf,wp,lp,save,gametype,vruns,hruns,wteam,lteam,line,batteries,lineups,box,pbp,season,homewon,timestamp,Latitude,Longitude,visdistancetraveled,homerestdays,visrestdays
218679,DET202410100,CLE,DET,DET05,20241010,0.0,6:08PM,night,9.0,NaN,True,NaN,185.0,44923,unknown,none,sunny,61,fromcf,2,kleir701,NaN,NaN,ticht901,fairc901,mahrn901,becka901,dejer901,wolfj901,gaddh001,brieb001,clase001,divisionseries,5,4,CLE,DET,y,both,y,y,y,2024,False,2024-10-10 18:08:00,42.339063,-83.048627,91.134933,1,1
218680,KCA202410100,NYA,KCA,KAN06,20241010,0.0,7:08PM,night,9.0,NaN,True,NaN,156.0,39012,unknown,none,sunny,78,rtol,6,butcp701,NaN,NaN,carlm901,barkl901,ortir901,hamaa901,blakr901,barrl901,coleg001,wachm001,weavl001,divisionseries,3,1,NYA,KCA,y,both,y,y,y,2024,False,2024-10-10 19:08:00,39.051604,-94.480149,1093.203095,1,1
218681,LAN202410110,SDN,LAN,LOS03,20241011,0.0,5:08PM,night,9.0,NaN,True,NaN,146.0,53183,unknown,none,cloudy,80,ltor,2,munse701,NaN,NaN,gibsh902,johna901,rippm901,libkj901,blasc901,belld901,yamay001,darvy001,treib001,divisionseries,0,2,LAN,SDN,y,both,y,y,y,2024,True,2024-10-11 17:08:00,34.073878,-118.239951,113.202533,2,2
218682,CLE202410120,DET,CLE,CLE08,20241012,0.0,1:08PM,day,9.0,NaN,True,NaN,188.0,34105,unknown,none,sunny,65,fromcf,7,maveb701,NaN,NaN,fairc901,mahrn901,becka901,dejer901,wolfj901,ticht901,herrt002,skubt001,clase001,divisionseries,3,7,CLE,DET,y,both,y,y,y,2024,True,2024-10-12 13:08:00,41.496005,-81.685326,91.134933,2,2
218683,LAN202410130,NYN,LAN,LOS03,20241013,0.0,5:15PM,night,9.0,NaN,True,NaN,172.0,53503,unknown,none,sunny,73,tocf,5,munse701,NaN,NaN,rehaj901,hudsm901,lentn901,millb901,rackd901,bakej902,flahj002,sengk001,NaN,lcs,0,9,LAN,NYN,y,both,y,y,y,2024,True,2024-10-13 17:15:00,34.073878,-118.239951,2452.947708,2,4
218684,LAN202410140,NYN,LAN,LOS03,20241014,0.0,1:08PM,day,9.0,NaN,True,NaN,207.0,52926,unknown,none,cloudy,73,tocf,6,eastr701,NaN,NaN,muchm901,lentn901,millb901,rackd901,bakej902,rehaj901,manas001,brasr001,diaze006,lcs,7,3,NYN,LAN,y,both,y,y,y,2024,False,2024-10-14 13:08:00,34.073878,-118.239951,2452.947708,1,1
218685,NYA202410140,CLE,NYA,NYC21,20241014,0.0,7:38PM,night,9.0,NaN,True,NaN,181.0,47264,unknown,none,cloudy,50,ltor,15,karph701,NaN,NaN,estam901,iassd901,segac901,porta901,viscj901,carav901,rodoc001,cobba001,weavl001,lcs,2,5,NYA,CLE,y,both,y,y,y,2024,True,2024-10-14 19:38:00,40.829586,-73.926413,406.072970,4,2
218686,NYA202410150,CLE,NYA,NYC21,20241015,0.0,7:38PM,night,9.0,NaN,True,NaN,206.0,47054,unknown,none,cloudy,52,ltor,2,kaplh701,NaN,NaN,morag901,segac901,porta901,viscj901,carav901,estam901,holmc001,bibet001,NaN,lcs,3,6,NYA,CLE,y,both,y,y,y,2024,True,2024-10-15 19:38:00,40.829586,-73.926413,406.072970,1,1
218687,NYN202410160,LAN,NYN,NYC20,20241016,0.0,8:08PM,night,9.0,NaN,True,NaN,191.0,43883,unknown,none,sunny,51,ltor,8,kaplh701,NaN,NaN,hudsm901,millb901,rackd901,bakej902,rehaj901,muchm901,kopem001,sevel001,NaN,lcs,8,0,LAN,NYN,y,both,y,y,y,2024,False,2024-10-16 20:08:00,40.757134,-73.845840,2452.947708,2,2
218688,CLE202410170,NYA,CLE,CLE08,20241017,0.0,5:08PM,night,9.0,NaN,True,NaN,232.0,32531,unknown,none,sunny,57,fromcf,5,libea701,NaN,NaN,iassd901,porta901,viscj901,carav901,estam901,morag901,avilp001,holmc001,NaN,lcs,5,7,CLE,NYA,y,both,y,y,y,2024,True,2024-10-17 17:08:00,41.496005,-81.685326,406.072970,2,2


## 6. Save to `.csv`

In [ ]:
all_games.to_csv('./gameinfo_cleaned.csv', index=False)